You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os
#"""
# !rm k -r
if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1349, done.
remote: Counting objects: 100% (798/798), done.
remote: Compressing objects: 100% (512/512), done.
remote: Total 1349 (delta 587), reused 491 (delta 285), pack-reused 551
Receiving objects: 100% (1349/1349), 15.00 MiB | 31.16 MiB/s, done.
Resolving deltas: 100% (943/943), done.
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=56793 sha256=177fdbe51d9e72ec3b2d3624b8842bc3067958a834de2a17c02effbbe9136ab5
  Stored in directory: /tmp/pip-ephem-wheel-cache-d80bpsnc/wheels/c1/8a/57/56dbba25eff58e52e536543

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.mobilenet_v3
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.6.0
Keras version: 2.6.0


In [ ]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [ ]:
# mixed_precision.set_global_policy('mixed_float16')

# Create folder structure with CIFAR-10 as png files.

In [ ]:
verbose=True
root_folder = 'cifar10-as-png'
data_dir = root_folder + '/train';
test_dir = root_folder + '/test';

x_train, y_train, x_test, y_test = cai.datasets.load_dataset(tf.keras.datasets.cifar10, verbose=verbose, lab=False, bipolar=False)

if not os.path.isdir(root_folder):
  os.mkdir(root_folder)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_train)*255, y_train, dest_folder_name=data_dir)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_test)*255, y_test, dest_folder_name=test_dir)

170508288/170498071 [==============================] - 2s 0us/step
train shape (50000, 32, 32, 3)
test shape (10000, 32, 32, 3)
Original channel  0  min: 0.0  max: 255.0
Original channel  1  min: 0.0  max: 255.0
Original channel  2  min: 0.0  max: 255.0
Loading RGB.
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0


In [ ]:
num_classes = 10
batch_size = 64
epochs = 50
target_size_x = 224
target_size_y = 224
seed = 12

In [ ]:
train_datagen = cai.util.create_image_generator(validation_split=0.1, rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.0, rescale=1./255)
test_datagen = cai.util.create_image_generator_no_augmentation(rescale=1./255)
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [ ]:
def work_on_kmobilenet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [13]:
      basefilename = 'JP36B11-mobilenet-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.mobilenet_v3.MobileNetV3Large(
          input_shape=input_shape,
          alpha=1.0,
          minimalistic=False,
          include_top=True,
          input_tensor=None,
          classes=num_classes,
          pooling=None,
          dropout_rate=0.2)
      else:
        model = cai.mobilenet_v3.kMobileNetV3Large(
          input_shape=input_shape,
          alpha=1.0,
          minimalistic=False,
          include_top=True,
          input_tensor=None,
          classes=num_classes,
          pooling=None,
          dropout_rate=0.2,
          kType=kType)

      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model):
        model.summary(line_length=180)
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="training",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        validation_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="validation",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow_from_directory(
            directory=test_dir,
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_kmobilenet(show_model=True, run_fit=False, test_results=False)

Running: JP36B11-mobilenet-CIFAR10-13
Model: "kMobilenetV3large-13"
____________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                               Output Shape                            Param #              Connected to                                                
input_1 (InputLayer)                                       [(None, 224, 224, 3)]                   0                                                                                
____________________________________________________________________________________________________________________________________________________________________________________
Conv_pad (ZeroPadding2D)                                   (None, 225, 225, 3)                     0                    input_1[0][0]                                               
___________________________

# Fitting

In [ ]:
work_on_kmobilenet(show_model=False, run_fit=True, test_results=True)

Running: JP36B11-mobilenet-CIFAR10-13
Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
704/704 [==============================] - 754s 951ms/step - loss: 1.5993 - accuracy: 0.4077 - val_loss: 2.4809 - val_accuracy: 0.1000

Epoch 00001: val_accuracy improved from -inf to 0.10000, saving model to JP36B11-mobilenet-CIFAR10-13-best_result.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
704/704 [==============================] - 650s 918ms/step - loss: 1.2522 - accuracy: 0.5524 - val_loss: 2.8786 - val_accuracy: 0.1000

Epoch 00002: val_accuracy did not improve from 0.10000
Epoch 3/50
704/704 [==============================] - 652s 920ms/step - loss: 1.0771 - accuracy: 0.6206 - val_loss: 3.3789 - val_accuracy: 0.1142

Epoch 00003: val_accuracy improved from 0.10000 to 0.11420, saving model to JP36B11-mobilenet-CIFAR10-13-best_result.hdf5
Epoch 4/50
704/704 [==============================] - 669s 937ms/step - loss: 0.9642 - accuracy: 0.6633 - val_loss: 3.0896 - val_accuracy: 0.1578

Epoch 00004: val_accuracy improved from 0.11420 to 0.15780, saving model to JP36B11-mobilenet-CIFAR10-13-best_result.hdf5
Epoch 5/50
704/704 [==============================] - 675s 946ms/step - loss: 0.8934 - accuracy: 0.6910 - val_loss: 1.9806 - val_accuracy: 0.4392

Epoch 00005: val_accuracy improved from 0.15780 to 0.43920, saving model to JP36B11-mobilenet-CIFAR10-13-best_res

# Test Results

In [ ]:
work_on_kmobilenet(show_model=False, run_fit=False, test_results=True)

Running: JP36B11-mobilenet-CIFAR10-13
Found 10000 images belonging to 10 classes.
Best Model Results: JP36B11-mobilenet-CIFAR10-13-best_result.hdf5
157/157 [==============================] - 29s 106ms/step - loss: 0.2406 - accuracy: 0.9326
loss 0.2405560314655304
acc 0.9326000213623047
Finished: JP36B11-mobilenet-CIFAR10-13
